In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import warnings
warnings.filterwarnings('ignore')
import transformers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Subset
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm
from transformers import AdamW


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import warnings
warnings.filterwarnings('ignore')
class AITextDataset(Dataset):
    def __init__(self, tokenizer,max_length, data_type ='train',model_type ='bert',file_path ="./dataset/train_set.json"):
        super(AITextDataset, self).__init__()
        global model
        model  = model_type
        self.path = file_path
        self.data = pd.read_json(self.path)
        self.tokenizer = tokenizer
        self.target = self.data.iloc[:,1]
        self.max_length = max_length
        self.data_type = data_type
        self.model_type = model_type
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        
        text = self.data.iloc[index,1]
        if self.model_type == 'bert':
            inputs = self.tokenizer.encode_plus(
                text ,
                None,
                padding = True,
                add_special_tokens = True,
                return_attention_mask = True,
                truncation = True,
                max_length = self.max_length,
            )
            ids = inputs["input_ids"]
            token_type_ids = inputs["token_type_ids"]
            mask = inputs["attention_mask"]
            if self.data_type == 'train':
                return {
                    'text_id': self.data.iloc[index,0],
                    'ids': torch.tensor(ids, dtype=torch.long),
                    'mask': torch.tensor(mask, dtype=torch.long),
                    'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),

                    'target': torch.tensor(self.data.iloc[index, 2],dtype=torch.long)
                    }
            else:
                return {
                    'text_id': self.data.iloc[index,0],
                    'ids': torch.tensor(ids, dtype=torch.long),
                    'mask': torch.tensor(mask, dtype=torch.long),
                    'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
                    }
        else:
            inputs = self.tokenizer.encode_plus(
                text ,
                None,
                padding = True,
                add_special_tokens = True,
                return_attention_mask = True,
                truncation = True,
                max_length = self.max_length,
            )
         
            ids = inputs["input_ids"]
    
            mask = inputs["attention_mask"]
            if self.data_type == 'train':
                return {
                    'text_id': self.data.iloc[index,0],
                    'ids': torch.tensor(ids, dtype=torch.long),
                    'mask': torch.tensor(mask, dtype=torch.long),
                    'target': torch.tensor(self.data.iloc[index, 2],dtype=torch.long)
                    }
            else:
                return {
                    'text_id': self.data.iloc[index,0],
                    'ids': torch.tensor(ids, dtype=torch.long),
                    'mask': torch.tensor(mask, dtype=torch.long)
                    }

    

    @staticmethod
    def collate_fn_train(data):
        ids =[torch.tensor(d['ids']) for d in data]
        masks =[torch.tensor(d['mask']) for d in data]
        if model =='bert':
            token_type_ids = [torch.tensor(d['token_type_ids']) for d in data]
        labels = [d['target'] for d in data]
        text_ids = [d['text_id'] for d in data]
        ids_ = pad_sequence(ids, batch_first=True)
        masks_ = pad_sequence(masks, batch_first=True)
        if model =='bert':
            token_type_ids_= pad_sequence(token_type_ids, batch_first=True)
        labels_ = torch.tensor(labels)
        text_id_ = torch.tensor(text_ids)
        if model == 'bert':
            return {
                'text_id': text_id_,
                'ids': ids_,
                'mask': masks_,
                'token_type_ids': token_type_ids_,
                'target': labels_
                }
        else:
            return {
                'text_id': text_id_,
                'ids': ids_,
                'mask': masks_,
                'target': labels_
                }

    @staticmethod
    def collate_fn_test(data):
        ids= [torch.tensor(d['ids']) for d in data]
        masks =[torch.tensor(d['mask']) for d in data]
        if model =='bert':
            token_type_ids = [torch.tensor(d['token_type_ids']) for d in data]       
        ids_ = pad_sequence(ids, batch_first=True)
        masks_ = pad_sequence(masks, batch_first=True)
        if model =='bert':
            token_type_ids_ = pad_sequence(token_type_ids, batch_first=True)
        text_ids = [d['text_id'] for d in data]
        text_id_ =  torch.tensor(text_ids)
        if model == 'bert':
            return {
                'text_id': text_id_,
                'ids': ids_,
                'mask': masks_,
                'token_type_ids': token_type_ids_
            }
        else:
            return {
                'text_id': text_id_,
                'ids': ids_,
                'mask': masks_
            }


     

In [ ]:

tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
PATH = "/content/drive/MyDrive/dataset"
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
arch ='bert'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
max_length =128
epochs = 50

In [ ]:
device

In [ ]:
def validation_loop(dataloader,model,loss_fn):
    model.eval()
    loop = tqdm(enumerate(dataloader),leave=False,total=len(dataloader))
    num_correct = 0
    target_count = 0
    total_loss  = 0
    with torch.no_grad():
        for batch, dl in loop:
            ids=dl['ids'].to(device)
            if arch =='bert':
                token_type_ids=dl['token_type_ids'].to(device)
            mask= dl['mask'].to(device)
            label=dl['target'].to(device)
            label = label.unsqueeze(1).to(device)
            if arch =='bert':
                output = model(ids,mask,token_type_ids)
            else:
                output = model(ids,mask)

            label = label.type_as(output) 
            loss = loss_fn(output,label)               
            output =output.cpu().detach().numpy()
            pred = np.where(output >= 0, 1, 0)
            target_count += label.size(0)
            

            # compute accuracy per batch
            num_correct+= sum(1 for a, b in zip(pred, label) if a[0] == b[0])
            total_loss+= loss.item() 
       
    print(f'Validation loss :{round(float(total_loss/len(dataloader)),3)}   with accuracy {round(float(100 * num_correct /target_count),3)}%')
    return float(100 * num_correct / target_count)
    

def training_loop(epochs,dataloader,val_dataloader,model,loss_fn,optimizer,scheduler):
    model.train()
    for  epoch in range(1,epochs+1):       
        loop=tqdm(enumerate(dataloader),leave=False,total=len(dataloader))
        total_loss  = 0
        num_correct = 0
        target_count = 0
        for batch, dl in loop:
            ids=dl['ids'].to(device)
            if arch =='bert':
                token_type_ids = dl['token_type_ids'].to(device)
            mask= dl['mask'].to(device)
            label=dl['target'].to(device)
            label = label.unsqueeze(1).to(device)

            optimizer.zero_grad()
            
            if arch =='bert':
                output = model(ids,mask,token_type_ids)
            else:
                output = model(ids,mask)
            label = label.type_as(output)

            loss = loss_fn(output,label)
            # back propagate
            loss.backward()  
            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)          
            optimizer.step()
            scheduler.step()
            
            output = output.cpu().detach().numpy()
            pred = np.where(output >= 0, 1, 0)

            target_count += label.size(0)
            
            # compute accuracy per batch
            num_correct+= sum(1 for a, b in zip(pred, label) if a[0] == b[0]) 
              
          
            total_loss+= loss.item() 
      
        print(f'Training loss :{round(float(total_loss/len(dataloader)),3)}   with accuracy {round(float(100 * num_correct /target_count),3)}%')
        if epoch%1 == 0:
           val_acc = validation_loop(val_dataloader,model,loss_fn)   
           # Show progress while training
           loop.set_description(f'Epochs={epoch}/{epochs}')
           # saved model 
           torch.save(model.state_dict(), PATH+'/ai-text-classifier-'+str(val_acc)+'.pth')
            

    return model



In [ ]:
class BertAITextDetector(nn.Module):
    def __init__(self,dropout= 0.20, base_model ='bert-base'):
        '''
           Bert model with a new fully connected layer for text classification
        '''
        super(BertAITextDetector, self).__init__()
        self.bert = baseModel(name=base_model).pretrained_model()
        self.dropout =  dropout
        self.fc = nn.Sequential(
          nn.Dropout(p = self.dropout),        
          nn.Linear(768,1)
        )
        self._init_weights()
        
    def forward(self,ids,mask,token_type_ids):
        _,o2= self.bert(ids,attention_mask=mask,token_type_ids=token_type_ids, return_dict=False)
        
        out= self.fc(o2)
        
        return out

    def _init_weights(self):
        """ Initialize the weights """
        for  m in self.fc:
            if isinstance(m, nn.Linear):
                m.weight.data.normal_(mean=0.0, std=0.02)
        
            if isinstance(m, nn.Linear) and m.bias is not None:
                m.bias.data.zero_()
    

class baseModel:
    def __init__(self, name ="bert-base"):
        self.name = name

    def pretrained_model(self):
        if self.name == 'bert-base':
            return transformers.BertModel.from_pretrained('bert-base-uncased')
        elif self.name =='bert-large':
            return transformers.BertModel.from_pretrained('bert-large-uncased')



In [ ]:
model_ = BertAITextDetector(dropout = 0.20)

In [ ]:
model_.cuda()

In [ ]:
from transformers import AdamW,get_linear_schedule_with_warmup
import torch.nn as nn

dataset = AITextDataset(tokenizer =tokenizer, max_length = 128, data_type ='train',model_type ='bert',file_path ='/content/drive/MyDrive/dataset/train_set.json')
dataset_size = len(dataset)
validation_split = 0.20
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
train_split = Subset(dataset, train_indices)

val_split = Subset(dataset, val_indices)
train_dataloader = torch.utils.data.DataLoader(train_split, batch_size=16, shuffle=True, num_workers=4,collate_fn = dataset.collate_fn_train)
val_dataloader = torch.utils.data.DataLoader(val_split, batch_size=16, shuffle=False, num_workers=4,collate_fn= dataset.collate_fn_train)
##Training parameters:
epochs = 5
# construct an optimizer
arch ='bert'
optimizer = AdamW(model_.parameters(), lr=2e-5,eps = 1e-08)

# Set up the learning rate scheduler
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
loss_fn = nn.BCEWithLogitsLoss()

model = training_loop(epochs, train_dataloader,val_dataloader, model_, loss_fn, optimizer,scheduler)

torch.save(model.state_dict(), PATH +'/ai-text-'+arch+str(epochs)+'-classifier-max-length-'+str(max_length)+'.pth')

In [ ]:
import csv
dataset = AITextDataset(tokenizer = tokenizer, max_length = max_length, data_type = 'test', model_type = 'bert', file_path = '/content/drive/MyDrive/dataset/test_set.json')
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 1, shuffle = False, collate_fn = dataset.collate_fn_test)
model_.load_state_dict(torch.load(PATH+'/ai-text-classifier-95.5.pth'))
model_.eval()
model_.to(device)

with torch.no_grad():
    with open(PATH+"/submission--1.csv", "w") as file:
        csv_out = csv.writer(file)
        csv_out.writerow(['id','label'])
        loop=tqdm(enumerate(dataloader),leave=False,total=len(dataloader))

        for i,(batch, dl) in enumerate(loop):
            ids = dl['ids'].to(device)
            if arch =='bert':
                token_type_ids=dl['token_type_ids'].to(device)
            mask= dl['mask'].to(device)
           
            if arch =='bert':
                output = model_(ids,mask,token_type_ids)
            else:
                 output=model_(ids,mask)
          
            output =output.logits.cpu().detach().numpy()
            pred = np.where(output >= 0, 1, 0)  
            row = pred.tolist()[0][0]
            csv_out.writerow([i, row])
 




In [ ]:
class RobertaAITextDetector(nn.Module):
    def __init__(self, dropout=0.3):
        '''
           Roberta model with new fully connected layer for text classification
        '''
        super(RobertaAITextDetector, self).__init__()    
        self.dropout_rate = dropout    
        self.bert = transformers.RobertaModel.from_pretrained('roberta-base')
        self.fc = nn.Sequential(
            nn.Linear(768,64),
            nn.ReLU(),
            nn.Linear(64,1)
        )
        self._init_weights()       
        
    def forward(self, input_ids, attention_mask):
        _, x2 = self.bert(input_ids=input_ids, attention_mask=attention_mask,return_dict=False)
        x = self.fc(x2)
        return x
        
    def _init_weights(self):
        """ Initialize the weights """
        for  m in self.fc:
            if isinstance(m, (nn.Linear,nn.LayerNorm)):
                m.weight.data.normal_(mean=0.0, std=0.02)
        
            if isinstance(m, (nn.Linear,nn.LayerNorm)) and m.bias is not None:
                m.bias.data.zero_()
    

In [ ]:
model_robert = RobertaAITextDetector(dropout = 0.20)
model_robert.cuda()

In [ ]:
tokenizer =transformers.RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
from transformers import AdamW,get_linear_schedule_with_warmup
import torch.nn as nn
PATH ='/content/drive/MyDrive/dataset/robert2'
dataset = AITextDataset(tokenizer =tokenizer, max_length = 128, data_type ='train',model_type ='robert',file_path ='/content/drive/MyDrive/dataset/train_set.json')
dataset_size = len(dataset)
validation_split = 0.20
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
train_split = Subset(dataset, train_indices)

val_split = Subset(dataset, val_indices)
train_dataloader = torch.utils.data.DataLoader(train_split, batch_size=16, shuffle=True, num_workers=4,collate_fn = dataset.collate_fn_train)
val_dataloader = torch.utils.data.DataLoader(val_split, batch_size=16, shuffle=False, num_workers=4,collate_fn= dataset.collate_fn_train)
##Training parameters:
epochs = 6
# construct an optimizer
arch ='robert'
optimizer = AdamW(model_robert.parameters(), lr=2e-5,eps = 1e-08)

# Set up the learning rate scheduler
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
loss_fn = nn.BCEWithLogitsLoss()

model = training_loop(epochs, train_dataloader,val_dataloader, model_robert, loss_fn, optimizer,scheduler)

torch.save(model.state_dict(), PATH +'/ai-text-'+arch+str(epochs)+'-classifier-max-length-'+str(max_length)+'.pth')

**Robert model inference -test set**

In [ ]:
import csv
dataset = AITextDataset(tokenizer = tokenizer, max_length = max_length, data_type = 'test', model_type = 'robe', file_path = '/content/drive/MyDrive/dataset/test_set.json')
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 1, shuffle = False, collate_fn = dataset.collate_fn_test)
# model_robert.load_state_dict(torch.load(PATH+'/ai-text-classifier-95.5.pth'))
model_robert.eval()
model_robert.to(device)

with torch.no_grad():
    with open(PATH+"/submission-robert2.csv", "w") as file:
        csv_out = csv.writer(file)
        csv_out.writerow(['id','label'])
        loop=tqdm(enumerate(dataloader),leave=False,total=len(dataloader))

        for i,(batch, dl) in enumerate(loop):
            ids = dl['ids'].to(device)
            if arch =='bert':
                token_type_ids=dl['token_type_ids'].to(device)
            mask= dl['mask'].to(device)
           
            if arch =='bert':
                output = model_robert(ids,mask,token_type_ids)
            else:
                 output=model_robert(ids,mask)
          
            output =output.cpu().detach().numpy()
            pred = np.where(output >= 0, 1, 0)  
            row = pred.tolist()[0][0]
            csv_out.writerow([i, row])
 




**GPT model for Text Classification**

In [ ]:
from transformers import GPT2Model
class GPTAITextDetector(nn.Module):
    def __init__(self, dropout=0.3):
        '''
           GPT2 model with new fully connected layer for text classification
        '''
        super(GPTAITextDetector, self).__init__()    
        self.dropout_rate = dropout 
        self.base =  GPT2Model.from_pretrained('gpt2')
        self.fc = nn.Sequential(
            nn.Linear(768,64),
            nn.ReLU(),
            nn.Linear(64,1)
        )
        
        self._init_weights()       
        
    def forward(self, input_ids, attention_mask):
        '''
        Inspire by :
                    https://github.com/huggingface/transformers/blob/v4.24.0/src/transformers/models/gpt2/modeling_gpt2.py#L1328
        
        '''
        x = self.base(input_ids=input_ids, attention_mask=attention_mask,return_dict=True)
        sequence = x[0]
   
        logits = self.fc(sequence)
        if input_ids is not None:
            batch_size, sequence_length = input_ids.shape[:2]

        if self.base.config.pad_token_id is None:
            sequence_lengths = -1
        else:
            if input_ids is not None:
                sequence_lengths = torch.ne(input_ids, self.base.config.pad_token_id).sum(-1) - 1
            else:
                sequence_lengths = -1
        
        pooled_logits = logits[torch.arange(batch_size, device=logits.device), sequence_lengths]
        return pooled_logits
        
    def _init_weights(self):
        """ Initialize the weights """
        for  m in self.fc:
            if isinstance(m, (nn.Linear,nn.LayerNorm)):
                m.weight.data.normal_(mean=0.0, std=0.02)
        
            if isinstance(m, (nn.Linear,nn.LayerNorm)) and m.bias is not None:
                m.bias.data.zero_()
    

In [ ]:
model_gpt = GPTAITextDetector(dropout = 0.20)
model_gpt.cuda()

PATH = '/content/drive/MyDrive/dataset/gpt'

In [ ]:
# Get model's tokenizer.
tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.padding_side = "left"

tokenizer.add_special_tokens({"cls_token": "[CLS]"})
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model_gpt.base.resize_token_embeddings(len(tokenizer))

In [ ]:
from transformers import AdamW,get_linear_schedule_with_warmup
import torch.nn as nn

dataset = AITextDataset(tokenizer =tokenizer, max_length = 128, data_type ='train',model_type ='gpt',file_path ='/content/drive/MyDrive/dataset/train_set.json')
dataset_size = len(dataset)
validation_split = 0.20
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
train_split = Subset(dataset, train_indices)

val_split = Subset(dataset, val_indices)
train_dataloader = torch.utils.data.DataLoader(train_split, batch_size=16, shuffle=True, num_workers=4,collate_fn = dataset.collate_fn_train)
val_dataloader = torch.utils.data.DataLoader(val_split, batch_size=16, shuffle=False, num_workers=4,collate_fn= dataset.collate_fn_train)
##Training parameters:
epochs = 5
# construct an optimizer
arch ='gpt'
optimizer = AdamW(model_gpt.parameters(), lr=2e-4,eps = 1e-08)

# Set up the learning rate scheduler
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
loss_fn = nn.BCEWithLogitsLoss()

model = training_loop(epochs, train_dataloader,val_dataloader, model_gpt, loss_fn, optimizer,scheduler)

torch.save(model.state_dict(), PATH +'/ai-text-'+arch+str(epochs)+'-classifier-max-length-'+str(max_length)+'.pth')

In [ ]:
import csv
dataset = AITextDataset(tokenizer = tokenizer, max_length = max_length, data_type = 'test', model_type = 'gpt', file_path = '/content/drive/MyDrive/dataset/test_set.json')
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 1, shuffle = False, collate_fn = dataset.collate_fn_test)

model_gpt.eval()
model_gpt.to(device)

with torch.no_grad():
    with open(PATH+"/submission-gpt.csv", "w") as file:
        csv_out = csv.writer(file)
        csv_out.writerow(['id','label'])
        loop=tqdm(enumerate(dataloader),leave=False,total=len(dataloader))

        for i,(batch, dl) in enumerate(loop):
            ids = dl['ids'].to(device)
            if arch =='bert':
                token_type_ids=dl['token_type_ids'].to(device)
            mask= dl['mask'].to(device)
           
            if arch =='bert':
                output = model_robert(ids,mask,token_type_ids)
            else:
                 output=model_robert(ids,mask)
          
            output =output.cpu().detach().numpy()
            pred = np.where(output >= 0, 1, 0)  
            row = pred.tolist()[0][0]
            csv_out.writerow([i, row])
 


